In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import requests

url = "https://air-quality.p.rapidapi.com/history/airquality"

querystring = {"lon":"-78.638","lat":"35.779"}

headers = {
	"x-rapidapi-key": "620a1c45b5msh05798e86e1c30b0p14f1dajsn981b24a1f455",
	"x-rapidapi-host": "air-quality.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json()['data'])

[{'aqi': 54, 'co': 107.4, 'datetime': '2024-12-24:15', 'no2': 21, 'o3': 31, 'pm10': 15, 'pm25': 14, 'so2': 19, 'timestamp_local': '2024-12-24T10:00:00', 'timestamp_utc': '2024-12-24T15:00:00', 'ts': 1735052400}, {'aqi': 61, 'co': 675, 'datetime': '2024-12-24:14', 'no2': 173, 'o3': 30, 'pm10': 19, 'pm25': 17.25, 'so2': 17.7, 'timestamp_local': '2024-12-24T09:00:00', 'timestamp_utc': '2024-12-24T14:00:00', 'ts': 1735048800}, {'aqi': 42, 'co': 564, 'datetime': '2024-12-24:13', 'no2': 172, 'o3': 29, 'pm10': 10, 'pm25': 10, 'so2': 1, 'timestamp_local': '2024-12-24T08:00:00', 'timestamp_utc': '2024-12-24T13:00:00', 'ts': 1735045200}, {'aqi': 62, 'co': 97.5, 'datetime': '2024-12-24:12', 'no2': 19, 'o3': 28, 'pm10': 18.3, 'pm25': 17.33, 'so2': 15, 'timestamp_local': '2024-12-24T07:00:00', 'timestamp_utc': '2024-12-24T12:00:00', 'ts': 1735041600}, {'aqi': 55, 'co': 96.3, 'datetime': '2024-12-24:11', 'no2': 18.7, 'o3': 5.2, 'pm10': 17.2, 'pm25': 14.33, 'so2': 0, 'timestamp_local': '2024-12-24T06

In [7]:
df=pd.DataFrame(response.json()['data'])

In [11]:
df.head()

,aqi,co,datetime,no2,o3,pm10,pm25,so2,timestamp_local,timestamp_utc,ts
0,54,107.4,2024-12-24:15,21.0,31.0,15.0,14.00,19.0,2024-12-24T10:00:00,2024-12-24T15:00:00,1735052400
1,61,675.0,2024-12-24:14,173.0,30.0,19.0,17.25,17.7,2024-12-24T09:00:00,2024-12-24T14:00:00,1735048800
2,42,564.0,2024-12-24:13,172.0,29.0,10.0,10.00,1.0,2024-12-24T08:00:00,2024-12-24T13:00:00,1735045200
3,62,97.5,2024-12-24:12,19.0,28.0,18.3,17.33,15.0,2024-12-24T07:00:00,2024-12-24T12:00:00,1735041600
4,55,96.3,2024-12-24:11,18.7,5.2,17.2,14.33,0.0,2024-12-24T06:00:00,2024-12-24T11:00:00,1735038000
